In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np


    

In [2]:
def pcafunction(x_data, y_data, num_componentes, name_column, name_index):

    # standarizar
    scaler_model = StandardScaler().fit(x_data)
    x = scaler_model.transform(x_data) 
 
    
    
    pca = PCA(n_components=num_componentes)
    principalComponents = pca.fit_transform(x)
    
    pca_df = pd.DataFrame(principalComponents, columns=[f'PC1_{name_column}', f'PC2_{name_column}'])
    pca_df['FECHA-HORA'] = y_data
    
    
    
    return pca,pca_df,scaler_model

In [3]:
def exogen_pca(dataframe, var_index):
   
    #variables
    dfvar = dataframe[['DEMANDA', 'MES', 'DIA', 'HORA','TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR', 'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB',
       'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO', 'TMIN-CUL', 'PREC_HMO_MM',
       'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM']]
    
    dfvar['FECHA-HORA']= dfvar.index
    
    #variables para pca
    y = dataframe.loc[:,[var_index]].values
    
    #PCA TEMP MAX
    x_tmax = dataframe.loc[:,'TMAX-CAB':'TMAX-CUL'].values 
    pca_tmax,pca_df_tmax, scaler_model_tmax = pcafunction(x_tmax, y, 2, 'TMAX', var_index)
    
    #PCA TEMP MIN
    x_tmin = dataframe.loc[:,'TMIN-CAB':'TMIN-CUL'].values 
    pca_tmin,pca_df_tmin, scaler_model_tmin = pcafunction(x_tmin, y, 2, 'TMIN', var_index)
    
    #PCA PRECIPITACION
    x_prec = dataframe.loc[:,'PREC_HMO_MM':'PREC_CUL_MM'].values 
    pca_prec,pca_df_prec,scaler_model_prec = pcafunction(x_prec, y, 2, 'PREC', var_index)
    
    #UNION DE LOS 3 DATAFRAMES
    pca_df = pd.merge(pca_df_tmax, pca_df_tmin, on=var_index, how='left')
    pca_df = pd.merge(pca_df, pca_df_prec, on=var_index, how='left')
    pca_df = pd.merge(pca_df, dfvar, left_on=var_index, right_on='FECHA-HORA', how='left')
    del pca_df['FECHA-HORA']
    
    return pca_df,pca_tmax,pca_tmin,pca_prec,scaler_model_tmax, scaler_model_tmin,scaler_model_prec

In [4]:
def new_dataframe(my_array,y_data,name_column, name_index ):
    df = pd.DataFrame(my_array, columns=[f'PC1_{name_column}', f'PC2_{name_column}'])
    df['FECHA-HORA'] = y_data
    
    return df

In [5]:
def transform_data(dataframe, var_index,pca_tmax,pca_tmin,pca_prec,scaler_model_tmax, scaler_model_tmin,scaler_model_prec):
    
    #variables
    dfvar = dataframe[['DEMANDA', 'MES', 'DIA', 'HORA']]
    dfvar['FECHA-HORA']= dfvar.index
    #variables y
    y = dfvar.index
    
    ##### PCA TEMP MAX #####
    #varibales de x
    x_tmax = dataframe.loc[:,'TMAX-CAB':'TMAX-CUL'].values 
    #standarizacion  TEMP MAX
    x_df_tmax = scaler_model_tmax.transform(x_tmax) 
    #trasnformacion
    tmax = pca_tmax.transform(x_df_tmax)
    
    ##### PCA TEMP MIN #####
    #varibales de x
    x_tmin = dataframe.loc[:,'TMIN-CAB':'TMIN-CUL'].values 
    #standarizacion  TEMP MAX
    x_df_tmin = scaler_model_tmin.transform(x_tmin) 
    #trasnformacion
    tmin = pca_tmin.transform(x_df_tmin)
    
    ##### PCA PRECIPITACION #####
    #varibales de x
    x_prec = dataframe.loc[:,'PREC_HMO_MM':'PREC_CUL_MM'].values 
    #standarizacion  TEMP MAX
    x_df_prec = scaler_model_prec.transform(x_prec) 
    #trasnformacion
    prec = pca_prec.transform(x_df_prec)
    
    df_tmax = new_dataframe(tmax,y,'TMAX', var_index )
    df_tmin = new_dataframe(tmin,y,'TMIN', var_index )
    df_prec = new_dataframe(prec,y,'PREC', var_index )
    
    #UNION DE LOS 3 DATAFRAMES
    pca_df = pd.merge(df_tmax, df_tmin, on=var_index, how='left')
    pca_df = pd.merge(pca_df, df_prec, on=var_index, how='left')
    pca_df = pd.merge(pca_df, dfvar, left_on=var_index, right_on='FECHA-HORA', how='left')
    #del pca_df['FECHA-HORA']

    return pca_df
    